In [1]:
import pandas as pd 
import os 
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/Superficie/2_ProteinOrtho/2_ProteinOrtho_IDs')
nod= pd.read_csv('Homo_nodos.csv')

Al archivo obtenido de ProteinOrtho (Ast-Hum) se agregaron los ID's (a2, b2) del proteoma obtenido a traves del NCBI

In [2]:
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/cueva/1_ProteinOrto/2_ProteinOrtho_IDs')
ortho = pd.read_csv('Ortho_Pach_homo.csv') 
ortho.head()

,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,Astyanax_pachon,KAG9259314.1,FKBP7,Homo_sapiens,NP_001397901.1,FKBP7,1.020000e-33,143.0,1.630000e-34,143.0
1,Astyanax_pachon,KAG9259355.1,GRIA4,Homo_sapiens,XP_054205792.1,GRIA2,1.210000e-145,516.0,2.910000e-146,516.0
2,Astyanax_pachon,KAG9259371.1,TPH2,Homo_sapiens,NP_775489.2,TPH2,1.660000e-230,799.0,2.010000e-231,799.0
3,Astyanax_pachon,KAG9259372.1,TBC1D15,Homo_sapiens,NP_001139685.2,TBC1D15,4.860000e-253,874.0,7.330000e-254,874.0
4,Astyanax_pachon,KAG9259372.1,TBC1D15,Homo_sapiens,XP_054228889.1,TBC1D15,9.710000e-253,873.0,1.450000e-253,873.0


In [3]:
ortho.shape

(56479, 10)

In [4]:
nod.head()

,Model_nodos,NCBI_Product_accesion
0,ACVRL1,NP_000011.2
1,ADRB2,NP_000015.2
2,ADRB3,NP_000016.1
3,ALDOB,NP_000026.2
4,APOC3,NP_000031.1


In [5]:
# Asegúrate de que no hay espacios extra o diferencias en los valores
nod['NCBI_Product_accesion'] = nod['NCBI_Product_accesion'].astype(str).str.strip()
ortho['b'] = ortho['b'].astype(str).str.strip()

# Filtrado: conservar solo las filas de 'ortho' que tengan coincidencias en 'nod'
filtrado = ortho[ortho['b'].isin(nod['NCBI_Product_accesion'])]

# Mostrar los primeros resultados
filtrado.head()

,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
2,Astyanax_pachon,KAG9259371.1,TPH2,Homo_sapiens,NP_775489.2,TPH2,1.660000e-230,799.0,2.010000e-231,799.0
46,Astyanax_pachon,KAG9259403.1,FGA,Homo_sapiens,XP_054205235.1,FGA,4.860000e-110,399.0,9.190000e-111,399.0
48,Astyanax_pachon,KAG9259404.1,FGA,Homo_sapiens,XP_054205235.1,FGA,1.950000e-109,397.0,3.680000e-110,397.0
49,Astyanax_pachon,KAG9259406.1,FGB,Homo_sapiens,NP_005132.2,FGB,9.200000e-158,557.0,1.430000e-158,557.0
63,Astyanax_pachon,KAG9259419.1,GDF5,Homo_sapiens,NP_000548.2,GDF5,3.240000e-126,452.0,5.900000e-127,452.0


In [6]:
filtrado.shape

(2053, 10)

In [7]:
#Unión de nodos del modelo con ortólogos
merged = pd.merge(nod, filtrado, left_on='NCBI_Product_accesion', right_on='b', how='left')
merged.head()

,Model_nodos,NCBI_Product_accesion,Ast,a,a2,Homo,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,ACVRL1,NP_000011.2,Astyanax_pachon,KAG9269753.1,ACVR1,Homo_sapiens,NP_000011.2,ACVRL1,3.080000e-153,542.0,4.790000e-154,542.0
1,ADRB2,NP_000015.2,Astyanax_pachon,KAG9265923.1,ADRB2,Homo_sapiens,NP_000015.2,ADRB2,7.070000e-114,411.0,1.070000e-114,411.0
2,ADRB2,NP_000015.2,Astyanax_pachon,KAG9272734.1,ADRB2,Homo_sapiens,NP_000015.2,ADRB2,5.400000e-116,418.0,8.360000e-117,418.0
3,ADRB3,NP_000016.1,Astyanax_pachon,KAG9270408.1,ADRB4C,Homo_sapiens,NP_000016.1,ADRB3,1.870000e-86,320.0,2.620000e-87,320.0
4,ADRB3,NP_000016.1,Astyanax_pachon,KAG9270409.1,ADRB4C,Homo_sapiens,NP_000016.1,ADRB3,1.950000e-86,320.0,2.620000e-87,320.0


In [8]:
# 4. Agrupar por nodo y accession ID — mantener lista de tuplas (ID, símbolo)
grouped = merged.groupby(['Model_nodos', 'b']).apply(
    lambda df: list(zip(df['a'], df['a2']))
).reset_index(name='Astyanax_ortologos')

# 5. Añadir símbolo del gen de los IDs de referencia
symbol_map = filtrado[['b', 'b2']].drop_duplicates().set_index('b')['b2']
grouped['Symbol'] = grouped['b'].map(symbol_map)

# 6. Renombrar columna 'a' a 'NCBI_Product_accesion' para mantener consistencia
grouped = grouped.rename(columns={'b': 'NCBI_Product_accesion'})

# 7. Reordenar columnas
grouped = grouped[['Model_nodos', 'NCBI_Product_accesion', 'Astyanax_ortologos']]
grouped.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_16096\3504083736.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = merged.groupby(['Model_nodos', 'b']).apply(


,Model_nodos,NCBI_Product_accesion,Astyanax_ortologos
0,A2M,NP_001334352.2,"[(KAG9265127.1, A2M), (KAG9265130.1, A2ML1), (..."
1,AATF,NP_036270.1,"[(KAG9281985.1, AATF)]"
2,ABCA1,NP_005493.2,"[(KAG9260005.1, ABCA1), (KAG9281151.1, ABCA1)]"
3,ABCA3,NP_001080.2,"[(KAG9264555.1, ABCA3)]"
4,ABCB1,NP_001335874.1,"[(KAG9283635.1, ABCB1)]"


In [9]:
grouped.to_csv('CV_Homo_Dic_Orto.csv')

In [10]:
merged.to_csv('CV_Homo_Dic_merged.csv')